## Lab 06: Data Lakehouse with Structured Streaming
This lab will help you learn to use many of the software libraries and programming techniques required to fulfill the requirements of the final end-of-session capstone project for course **DS-2002: Data Systems**. The spirit of the project is to provide a capstone challenge that requires students to demonstrate a practical and functional understanding of each of the data systems and architectural principles covered throughout the session.

**These include:**
- Relational Database Management Systems (e.g., MySQL, Microsoft SQL Server, Oracle, IBM DB2)
  - Online Transaction Processing Systems (OLTP): *Optimized for High-Volume Write Operations; Normalized to 3rd Normal Form.*
  - Online Analytical Processing Systems (OLAP): *Optimized for Read/Aggregation Operations; Dimensional Model (i.e, Star Schema)*
- NoSQL *(Not Only SQL)* Systems (e.g., MongoDB, CosmosDB, Cassandra, HBase, Redis)
- File System *(Data Lake)* Source Systems (e.g., AWS S3, Microsoft Azure Data Lake Storage)
  - Various Datafile Formats (e.g., JSON, CSV, Parquet, Text, Binary)
- Massively Parallel Processing *(MPP)* Data Integration Systems (e.g., Apache Spark, Databricks)
- Data Integration Patterns (e.g., Extract-Transform-Load, Extract-Load-Transform, Extract-Load-Transform-Load, Lambda & Kappa Architectures)

### Section I: Prerequisites

#### 1.0. Import Required Libraries

In [0]:
import os
import json
import pymongo
import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

#### 2.0. Instantiate Global Variables

In [0]:
# Azure MySQL Server Connection Information ###################
jdbc_hostname = "yvf7ua-mysql.mysql.database.azure.com"
jdbc_port = 3306
src_database = "northwind_dw2"

connection_properties = {
  "user" : "yvf7ua",
  "password" : "Passw0rd123",
  "driver" : "org.mariadb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "clustermidterm.7rtqy6x"
atlas_database_name = "northwind_dw2"
atlas_user_name = "yvf7ua"
atlas_password = "Passw0rd123"

# Data Files (JSON) Information ###############################
dst_database = "northwind_dlh"

base_dir = "dbfs:/FileStore/ds2002-lab06"
database_dir = f"{base_dir}/{dst_database}"

data_dir = f"{base_dir}/source_data"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"

orders_stream_dir = f"{stream_dir}/orders"
purchase_orders_stream_dir = f"{stream_dir}/purchase_orders"
inventory_trans_stream_dir = f"{stream_dir}/inventory_transactions"

orders_output_bronze = f"{database_dir}/fact_orders/bronze"
orders_output_silver = f"{database_dir}/fact_orders/silver"
orders_output_gold   = f"{database_dir}/fact_orders/gold"

purchase_orders_output_bronze = f"{database_dir}/fact_purchase_orders/bronze"
purchase_orders_output_silver = f"{database_dir}/fact_purchase_orders/silver"
purchase_orders_output_gold   = f"{database_dir}/fact_purchase_orders/gold"

inventory_trans_output_bronze = f"{database_dir}/fact_inventory_transactions/bronze"
inventory_trans_output_silver = f"{database_dir}/fact_inventory_transactions/silver"
inventory_trans_output_gold   = f"{database_dir}/fact_inventory_transactions/gold"

# Delete the Streaming Files ################################## 
dbutils.fs.rm(f"{database_dir}/fact_orders", True) 
dbutils.fs.rm(f"{database_dir}/fact_purchase_orders", True) 
dbutils.fs.rm(f"{database_dir}/fact_inventory_transactions", True)

# Delete the Database Files ###################################
dbutils.fs.rm(database_dir, True)

Out[2]: True

#### 3.0. Define Global Functions

In [0]:
# ######################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
# ######################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

# ######################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
# ######################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

### Section II: Populate Dimensions by Ingesting Reference (Cold-path) Data 
#### 1.0. Fetch Reference Data From an Azure MySQL Database
##### 1.1. Create a New Databricks Metadata Database.

In [0]:
%sql
DROP DATABASE IF EXISTS northwind_dlh CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS northwind_dlh
COMMENT "DS-2002 Lab 06 Database"
LOCATION "dbfs:/FileStore/ds2002-lab06/northwind_dlh"
WITH DBPROPERTIES (contains_pii = true, purpose = "DS-2002 Lab 6.0");

##### 1.2. Create a New Table that Sources Date Dimension Data from a Table in an Azure MySQL database.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://yvf7ua-mysql.mysql.database.azure.com:3306/northwind_dw2",
  dbtable "dim_date",
  user "yvf7ua",
  password "Passw0rd123"
)

In [0]:
%sql
USE DATABASE northwind_dlh;

CREATE OR REPLACE TABLE northwind_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/ds2002-lab06/northwind_dlh/dim_date"
AS SELECT * FROM view_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.dim_date;

col_name,data_type,comment
date_key,int,null
full_date,date,null
date_name,string,null
date_name_us,string,null
date_name_eu,string,null
day_of_week,int,null
day_name_of_week,string,null
day_of_month,int,null
day_of_year,int,null
weekday_weekend,string,null


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_date LIMIT 5

date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,week_of_year,month_name,month_of_year,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


##### 1.3. Create a New Table that Sources Product Dimension Data from an Azure MySQL database.

In [0]:
%sql
-- Create a Temporary View named "view_product" that extracts data from your MySQL Northwind database.
CREATE OR REPLACE TEMPORARY VIEW view_product
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://yvf7ua-mysql.mysql.database.azure.com:3306/northwind_dw2",
  dbtable "dim_products",
  user "yvf7ua",
  password "Passw0rd123"
)

In [0]:
%sql
USE DATABASE northwind_dlh;
-- Create a new table named "northwind_dlh.dim_product" using data from the view named "view_product"
CREATE OR REPLACE TABLE northwind_dlh.dim_product
COMMENT "Product Dimension Table"
LOCATION "dbfs:/FileStore/ds2002-lab06/northwind_dlh/dim_products"
AS SELECT * FROM view_product

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.dim_product;

col_name,data_type,comment
product_key,int,null
product_code,string,null
product_name,string,null
standard_cost,"decimal(19,4)",null
list_price,"decimal(19,4)",null
reorder_level,int,null
target_level,int,null
quantity_per_unit,string,null
discontinued,boolean,null
minimum_reorder_quantity,int,null


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_product LIMIT 5

product_key,product_code,product_name,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category
1,NWTB-1,Northwind Traders Chai,13.5000,18.0000,10,40,10 boxes x 20 bags,false,10,Beverages
3,NWTCO-3,Northwind Traders Syrup,7.5000,10.0000,25,100,12 - 550 ml bottles,false,25,Condiments
4,NWTCO-4,Northwind Traders Cajun Seasoning,16.5000,22.0000,10,40,48 - 6 oz jars,false,10,Condiments
5,NWTO-5,Northwind Traders Olive Oil,16.0125,21.3500,10,40,36 boxes,false,10,Oil
6,NWTJP-6,Northwind Traders Boysenberry Spread,18.7500,25.0000,25,100,12 - 8 oz jars,false,25,"Jams, Preserves"


#### 2.0. Fetch Reference Data from a MongoDB Atlas Database
##### 2.1. View the Data Files on the Databricks File System

In [0]:
display(dbutils.fs.ls(batch_dir))

path,name,size,modificationTime
dbfs:/FileStore/ds2002-lab06/source_data/batch/Northwind_DimCustomers.json,Northwind_DimCustomers.json,10476,1682297044000
dbfs:/FileStore/ds2002-lab06/source_data/batch/Northwind_DimEmployees.csv,Northwind_DimEmployees.csv,2164,1682297043000
dbfs:/FileStore/ds2002-lab06/source_data/batch/Northwind_DimInvoices.json,Northwind_DimInvoices.json,6263,1682297043000
dbfs:/FileStore/ds2002-lab06/source_data/batch/Northwind_DimShippers.csv,Northwind_DimShippers.csv,262,1682297044000
dbfs:/FileStore/ds2002-lab06/source_data/batch/Northwind_DimSuppliers.json,Northwind_DimSuppliers.json,1480,1682297043000


##### 2.2. Create a New MongoDB Database, and Load JSON Data Into a New MongoDB Collection
**NOTE:** The following cell **can** be run more than once because the **set_mongo_collection()** function **is** idempotent.

In [0]:
source_dir = '/dbfs/FileStore/ds2002-lab06/source_data/batch'
json_files = {"customers" : 'Northwind_DimCustomers.json', "suppliers" : 'Northwind_DimSuppliers.json', "invoices" : 'Northwind_DimInvoices.json'}

set_mongo_collection(atlas_user_name, atlas_password, atlas_cluster_name, atlas_database_name, source_dir, json_files) 

Out[15]: <pymongo.results.InsertManyResult at 0x7f0db87d6440>

##### 2.3.1. Fetch Customer Dimension Data from the New MongoDB Collection

In [0]:
%scala
import com.mongodb.spark._

val df_customer = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "northwind_dw2").option("collection", "customers").load()
.select("customer_key","company","last_name","first_name","job_title","business_phone","fax_number","address","city","state_province","zip_postal_code","country_region")

display(df_customer)

customer_key,company,last_name,first_name,job_title,business_phone,fax_number,address,city,state_province,zip_postal_code,country_region
1,Company A,Bedecs,Anna,Owner,(123)555-0100,(123)555-0101,123 1st Street,Seattle,WA,99999,USA
2,Company B,Gratacos Solsona,Antonio,Owner,(123)555-0100,(123)555-0101,123 2nd Street,Boston,MA,99999,USA
3,Company C,Axen,Thomas,Purchasing Representative,(123)555-0100,(123)555-0101,123 3rd Street,Los Angelas,CA,99999,USA
4,Company D,Lee,Christina,Purchasing Manager,(123)555-0100,(123)555-0101,123 4th Street,New York,NY,99999,USA
5,Company E,O’Donnell,Martin,Owner,(123)555-0100,(123)555-0101,123 5th Street,Minneapolis,MN,99999,USA
6,Company F,Pérez-Olaeta,Francisco,Purchasing Manager,(123)555-0100,(123)555-0101,123 6th Street,Milwaukee,WI,99999,USA
7,Company G,Xie,Ming-Yang,Owner,(123)555-0100,(123)555-0101,123 7th Street,Boise,ID,99999,USA
8,Company H,Andersen,Elizabeth,Purchasing Representative,(123)555-0100,(123)555-0101,123 8th Street,Portland,OR,99999,USA
9,Company I,Mortensen,Sven,Purchasing Manager,(123)555-0100,(123)555-0101,123 9th Street,Salt Lake City,UT,99999,USA
10,Company J,Wacker,Roland,Purchasing Manager,(123)555-0100,(123)555-0101,123 10th Street,Chicago,IL,99999,USA


In [0]:
%scala
df_customer.printSchema()

root
-- customer_key: integer (nullable = true)
-- company: string (nullable = true)
-- last_name: string (nullable = true)
-- first_name: string (nullable = true)
-- job_title: string (nullable = true)
-- business_phone: string (nullable = true)
-- fax_number: string (nullable = true)
-- address: string (nullable = true)
-- city: string (nullable = true)
-- state_province: string (nullable = true)
-- zip_postal_code: string (nullable = true)
-- country_region: string (nullable = true)

##### 2.3.2. Use the Spark DataFrame to Create a New Customer Dimension Table in the Databricks Metadata Database (northwind_dlh)

In [0]:
%scala
df_customer.write.format("delta").mode("overwrite").saveAsTable("northwind_dlh.dim_customer")

In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.dim_customer

col_name,data_type,comment
customer_key,int,null
company,string,null
last_name,string,null
first_name,string,null
job_title,string,null
business_phone,string,null
fax_number,string,null
address,string,null
city,string,null
state_province,string,null


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_customer LIMIT 5

customer_key,company,last_name,first_name,job_title,business_phone,fax_number,address,city,state_province,zip_postal_code,country_region
1,Company A,Bedecs,Anna,Owner,(123)555-0100,(123)555-0101,123 1st Street,Seattle,WA,99999,USA
2,Company B,Gratacos Solsona,Antonio,Owner,(123)555-0100,(123)555-0101,123 2nd Street,Boston,MA,99999,USA
3,Company C,Axen,Thomas,Purchasing Representative,(123)555-0100,(123)555-0101,123 3rd Street,Los Angelas,CA,99999,USA
4,Company D,Lee,Christina,Purchasing Manager,(123)555-0100,(123)555-0101,123 4th Street,New York,NY,99999,USA
5,Company E,O’Donnell,Martin,Owner,(123)555-0100,(123)555-0101,123 5th Street,Minneapolis,MN,99999,USA


##### 2.4.1 Fetch Supplier Dimension Data from the New MongoDB Collection

In [0]:
%scala
val df_supplier = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "northwind_dw2").option("collection", "suppliers").load()
.select("supplier_key","company","last_name","first_name","job_title")

display(df_supplier)

supplier_key,company,last_name,first_name,job_title
1,Supplier A,Andersen,Elizabeth A.,Sales Manager
2,Supplier B,Weiler,Cornelia,Sales Manager
3,Supplier C,Kelley,Madeleine,Sales Representative
4,Supplier D,Sato,Naoki,Marketing Manager
5,Supplier E,Hernandez-Echevarria,Amaya,Sales Manager
6,Supplier F,Hayakawa,Satomi,Marketing Assistant
7,Supplier G,Glasson,Stuart,Marketing Manager
8,Supplier H,Dunton,Bryn Paul,Sales Representative
9,Supplier I,Sandberg,Mikael,Sales Manager
10,Supplier J,Sousa,Luis,Sales Manager


In [0]:
%scala
df_supplier.printSchema()

root
-- supplier_key: integer (nullable = true)
-- company: string (nullable = true)
-- last_name: string (nullable = true)
-- first_name: string (nullable = true)
-- job_title: string (nullable = true)

##### 2.4.2. Use the Spark DataFrame to Create a New Suppliers Dimension Table in the Databricks Metadata Database (northwind_dlh)

In [0]:
%scala
df_supplier.write.format("delta").mode("overwrite").saveAsTable("northwind_dlh.dim_supplier")

In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.dim_supplier

col_name,data_type,comment
supplier_key,int,null
company,string,null
last_name,string,null
first_name,string,null
job_title,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,northwind_dlh,
Table,dim_supplier,


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_supplier LIMIT 5

supplier_key,company,last_name,first_name,job_title
1,Supplier A,Andersen,Elizabeth A.,Sales Manager
2,Supplier B,Weiler,Cornelia,Sales Manager
3,Supplier C,Kelley,Madeleine,Sales Representative
4,Supplier D,Sato,Naoki,Marketing Manager
5,Supplier E,Hernandez-Echevarria,Amaya,Sales Manager


##### 2.5.1 Fetch Invoice Dimension Data from teh New MongoDB Collection

In [0]:
%scala
val df_invoice = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "northwind_dw2").option("collection", "invoices").load()
.select("invoice_key","order_key", "invoice_date", "due_date", "tax", "shipping", "amount_due")

display(df_invoice)

invoice_key,order_key,invoice_date,due_date,tax,shipping,amount_due
5,31,2006-03-22 16:08:59,2006-04-04 11:43:08,0,0,0
6,32,2006-03-22 16:10:27,2006-04-04 11:43:08,0,0,0
7,40,2006-03-24 10:41:41,2006-04-04 11:43:08,0,0,0
8,39,2006-03-24 10:55:46,2006-04-04 11:43:08,0,0,0
9,38,2006-03-24 10:56:57,2006-04-04 11:43:08,0,0,0
10,37,2006-03-24 10:57:38,2006-04-04 11:43:08,0,0,0
11,36,2006-03-24 10:58:40,2006-04-04 11:43:08,0,0,0
12,35,2006-03-24 10:59:41,2006-04-04 11:43:08,0,0,0
13,34,2006-03-24 11:00:55,2006-04-04 11:43:08,0,0,0
14,33,2006-03-24 11:02:02,2006-04-04 11:43:08,0,0,0


In [0]:
%scala
df_invoice.printSchema()

root
-- invoice_key: integer (nullable = true)
-- order_key: integer (nullable = true)
-- invoice_date: string (nullable = true)
-- due_date: string (nullable = true)
-- tax: integer (nullable = true)
-- shipping: integer (nullable = true)
-- amount_due: integer (nullable = true)

##### 2.5.2. Use the Spark DataFrame to Create a New Invoices Dimension Table in the Databricks Metadata Database (northwind_dlh)

In [0]:
%scala
df_invoice.write.format("delta").mode("overwrite").saveAsTable("northwind_dlh.dim_invoice")

In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.dim_invoice

col_name,data_type,comment
invoice_key,int,null
order_key,int,null
invoice_date,string,null
due_date,string,null
tax,int,null
shipping,int,null
amount_due,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_invoice LIMIT 5

invoice_key,order_key,invoice_date,due_date,tax,shipping,amount_due
5,31,2006-03-22 16:08:59,2006-04-04 11:43:08,0,0,0
6,32,2006-03-22 16:10:27,2006-04-04 11:43:08,0,0,0
7,40,2006-03-24 10:41:41,2006-04-04 11:43:08,0,0,0
8,39,2006-03-24 10:55:46,2006-04-04 11:43:08,0,0,0
9,38,2006-03-24 10:56:57,2006-04-04 11:43:08,0,0,0


#### 3.0. Fetch Data from a File System
##### 3.1. Use PySpark to Read From a CSV File

In [0]:
employee_csv = f"{batch_dir}/Northwind_DimEmployees.csv"

df_employee = spark.read.format('csv').options(header='true', inferSchema='true').load(employee_csv)
display(df_employee)

employee_key,company,last_name,first_name,email_address,job_title,business_phone,home_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page
1,Northwind Traders,Freehafer,Nancy,nancy@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 1st Avenue,Seattle,WA,99999,USA,#http://northwindtraders.com#
2,Northwind Traders,Cencini,Andrew,andrew@northwindtraders.com,"Vice President, Sales",(123)555-0100,(123)555-0102,(123)555-0103,123 2nd Avenue,Bellevue,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
3,Northwind Traders,Kotas,Jan,jan@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 3rd Avenue,Redmond,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
4,Northwind Traders,Sergienko,Mariya,mariya@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 4th Avenue,Kirkland,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
5,Northwind Traders,Thorpe,Steven,steven@northwindtraders.com,Sales Manager,(123)555-0100,(123)555-0102,(123)555-0103,123 5th Avenue,Seattle,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
6,Northwind Traders,Neipper,Michael,michael@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 6th Avenue,Redmond,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
7,Northwind Traders,Zare,Robert,robert@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 7th Avenue,Seattle,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
8,Northwind Traders,Giussani,Laura,laura@northwindtraders.com,Sales Coordinator,(123)555-0100,(123)555-0102,(123)555-0103,123 8th Avenue,Redmond,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
9,Northwind Traders,Hellung-Larsen,Anne,anne@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 9th Avenue,Seattle,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#


In [0]:
df_employee.printSchema()

root
 |-- employee_key: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- email_address: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- business_phone: string (nullable = true)
 |-- home_phone: string (nullable = true)
 |-- fax_number: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- zip_postal_code: integer (nullable = true)
 |-- country_region: string (nullable = true)
 |-- web_page: string (nullable = true)



In [0]:
df_employee.write.format("delta").mode("overwrite").saveAsTable("northwind_dlh.dim_employee")

In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.dim_employee;

col_name,data_type,comment
employee_key,int,null
company,string,null
last_name,string,null
first_name,string,null
email_address,string,null
job_title,string,null
business_phone,string,null
home_phone,string,null
fax_number,string,null
address,string,null


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_employee LIMIT 5;

employee_key,company,last_name,first_name,email_address,job_title,business_phone,home_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page
1,Northwind Traders,Freehafer,Nancy,nancy@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 1st Avenue,Seattle,WA,99999,USA,#http://northwindtraders.com#
2,Northwind Traders,Cencini,Andrew,andrew@northwindtraders.com,"Vice President, Sales",(123)555-0100,(123)555-0102,(123)555-0103,123 2nd Avenue,Bellevue,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
3,Northwind Traders,Kotas,Jan,jan@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 3rd Avenue,Redmond,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
4,Northwind Traders,Sergienko,Mariya,mariya@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 4th Avenue,Kirkland,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#
5,Northwind Traders,Thorpe,Steven,steven@northwindtraders.com,Sales Manager,(123)555-0100,(123)555-0102,(123)555-0103,123 5th Avenue,Seattle,WA,99999,USA,http://northwindtraders.com#http://northwindtraders.com/#


##### 3.2 Use PySpark to Read Shipper Dimension Data from CSV File

In [0]:
shipper_csv = f"{batch_dir}/Northwind_DimShippers.csv"

df_shipper = spark.read.format('csv').options(header='true', inferSchema='true').load(shipper_csv)
display(df_shipper)

shipper_key,company,address,city,state_province,zip_postal_code,country_region
1,Shipping Company A,123 Any Street,Memphis,TN,99999,USA
2,Shipping Company B,123 Any Street,Memphis,TN,99999,USA
3,Shipping Company C,123 Any Street,Memphis,TN,99999,USA


In [0]:
df_shipper.printSchema()

root
 |-- shipper_key: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- zip_postal_code: integer (nullable = true)
 |-- country_region: string (nullable = true)



In [0]:
df_shipper.write.format("delta").mode("overwrite").saveAsTable("northwind_dlh.dim_shipper")

In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.dim_shipper;

col_name,data_type,comment
shipper_key,int,null
company,string,null
address,string,null
city,string,null
state_province,string,null
zip_postal_code,int,null
country_region,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
%sql
SELECT * FROM northwind_dlh.dim_shipper LIMIT 5;

shipper_key,company,address,city,state_province,zip_postal_code,country_region
1,Shipping Company A,123 Any Street,Memphis,TN,99999,USA
2,Shipping Company B,123 Any Street,Memphis,TN,99999,USA
3,Shipping Company C,123 Any Street,Memphis,TN,99999,USA


##### Verify Dimension Tables

In [0]:
%sql
USE northwind_dlh;
SHOW TABLES

database,tableName,isTemporary
northwind_dlh,dim_customer,false
northwind_dlh,dim_date,false
northwind_dlh,dim_employee,false
northwind_dlh,dim_invoice,false
northwind_dlh,dim_product,false
northwind_dlh,dim_shipper,false
northwind_dlh,dim_supplier,false
,view_date,true
,view_product,true


### Section III: Integrate Reference Data with Real-Time Data
#### 6.0. Use AutoLoader to Process Streaming (Hot Path) Orders Fact Data 
##### 6.1. Bronze Table: Process 'Raw' JSON Data

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 .option("cloudFiles.schemaHints", "fact_order_key BIGINT")
 .option("cloudFiles.schemaHints", "order_key BIGINT")
 .option("cloudFiles.schemaHints", "employee_key BIGINT")
 .option("cloudFiles.schemaHints", "customer_key BIGINT") 
 .option("cloudFiles.schemaHints", "product_key BIGINT")
 .option("cloudFiles.schemaHints", "shipper_key DECIMAL")
 .option("cloudFiles.schemaHints", "order_date_key DECIMAL")
 .option("cloudFiles.schemaHints", "paid_date_key DECIMAL")
 .option("cloudFiles.schemaHints", "shipped_date_key DECIMAL") 
 .option("cloudFiles.schemaHints", "quantity DECIMAL")
 .option("cloudFiles.schemaHints", "unit_price DECIMAL")
 .option("cloudFiles.schemaHints", "discount DECIMAL")
 .option("cloudFiles.schemaHints", "shipping_fee DECIMAL")
 .option("cloudFiles.schemaHints", "taxes DECIMAL")
 .option("cloudFiles.schemaHints", "tax_rate DECIMAL")
 .option("cloudFiles.schemaHints", "payment_type STRING")
 .option("cloudFiles.schemaHints", "order_status STRING")
 .option("cloudFiles.schemaHints", "order_details_status STRING")
 .option("cloudFiles.schemaLocation", orders_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(orders_stream_dir)
 .createOrReplaceTempView("orders_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW orders_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM orders_raw_tempview
)

In [0]:
%sql
SELECT * FROM orders_bronze_tempview

customer_key,discount,employee_key,fact_order_key,order_date_key,order_details_status,order_key,order_status,paid_date_key,payment_type,product_key,quantity,shipped_date_key,shipper_key,shipping_fee,tax_rate,taxes,unit_price,_rescued_data,receipt_time,source_file
9,0,7,39,20060605,Invoiced,73,Closed,20060605,Check,41,10,20060605,1,100,0,0,9.65,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
28,0,1,40,20060607,Invoiced,72,Closed,20060607,Credit Card,43,5,20060607,3,40,0,0,46.0,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
1,0,1,41,20060524,Invoiced,71,New,null,null,40,40,null,3,0,0,0,18.4,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
11,0,1,42,20060524,Invoiced,70,New,null,null,8,20,null,3,0,0,0,40.0,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
10,0,1,43,20060524,Invoiced,69,New,null,null,80,15,null,1,0,0,0,3.5,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
10,0,4,44,20060524,Invoiced,67,Closed,20060524,Credit Card,74,20,20060524,2,9,0,0,10.0,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
8,0,6,45,20060430,Invoiced,60,Closed,20060430,Credit Card,72,40,20060430,3,50,0,0,34.8,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
3,0,4,46,20060425,Invoiced,63,Closed,20060425,Cash,3,50,20060425,2,7,0,0,10.0,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
3,0,4,47,20060425,Invoiced,63,Closed,20060425,Cash,8,3,20060425,2,7,0,0,40.0,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json
4,0,3,48,20060422,Invoiced,58,Closed,20060422,Credit Card,20,40,20060422,1,5,0,0,81.0,null,2023-04-24T18:13:04.507+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders03.json


In [0]:
(spark.table("orders_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{orders_output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_bronze"))

Out[36]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0dc0151f40>

##### 6.2. Silver Table: Include Reference Data

In [0]:
(spark.readStream
  .table("fact_orders_bronze")
  .createOrReplaceTempView("orders_silver_tempview"))

In [0]:
%sql
SELECT * FROM orders_silver_tempview

customer_key,discount,employee_key,fact_order_key,order_date_key,order_details_status,order_key,order_status,paid_date_key,payment_type,product_key,quantity,shipped_date_key,shipper_key,shipping_fee,tax_rate,taxes,unit_price,_rescued_data,receipt_time,source_file
28,0,1,20,20060407,Invoiced,45,Closed,20060407,Credit Card,40,50,20060407,3,40,0,0,18.4,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
9,0,7,21,20060405,Invoiced,46,Closed,20060405,Check,57,100,20060405,1,100,0,0,19.5,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
9,0,7,22,20060405,Invoiced,46,Closed,20060405,Check,72,50,20060405,1,100,0,0,34.8,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
6,0,6,23,20060408,Invoiced,47,Closed,20060408,Credit Card,34,300,20060408,2,300,0,0,14.0,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
8,0,4,24,20060405,Invoiced,48,Closed,20060405,Check,8,25,20060405,2,50,0,0,40.0,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
8,0,4,25,20060405,Invoiced,48,Closed,20060405,Check,19,25,20060405,2,50,0,0,9.2,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
25,0,9,26,20060405,Invoiced,50,Closed,20060405,Cash,21,20,20060405,1,5,0,0,10.0,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
26,0,9,27,20060405,Invoiced,51,Closed,20060405,Credit Card,5,25,20060405,3,60,0,0,21.35,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
26,0,9,28,20060405,Invoiced,51,Closed,20060405,Credit Card,41,30,20060405,3,60,0,0,9.65,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json
26,0,9,29,20060405,Invoiced,51,Closed,20060405,Credit Card,40,30,20060405,3,60,0,0,18.4,null,2023-04-24T18:13:07.360+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/orders/Northwind_FactOrders02.json


In [0]:
%sql
DESCRIBE EXTENDED orders_silver_tempview

col_name,data_type,comment
customer_key,bigint,null
discount,bigint,null
employee_key,bigint,null
fact_order_key,bigint,null
order_date_key,bigint,null
order_details_status,string,null
order_key,bigint,null
order_status,string,null
paid_date_key,bigint,null
payment_type,string,null


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_orders_silver_tempview AS (
  SELECT o.fact_order_key,
      o.order_key,
      o.employee_key,
      e.last_name AS employee_last_name,
      e.first_name AS employee_first_name,
      e.job_title AS employee_job_title,
      e.company AS employee_company,
      o.customer_key,
      c.last_name AS customer_last_name,
      c.first_name AS customer_first_name,
      o.product_key,
      p.product_code,
      p.product_name,
      p.standard_cost AS product_standard_cost,
      p.list_price AS product_list_price,
      p.category AS product_category,
      o.shipper_key,
      s.company AS shipper_company,
      s.state_province AS shipper_state_province,
      s.country_region AS shipper_country_region,
      o.order_date_key,
      od.day_name_of_week AS order_day_name_of_week,
      od.day_of_month AS order_day_of_month,
      od.weekday_weekend AS order_weekday_weekend,
      od.month_name AS order_month_name,
      od.calendar_quarter AS order_quarter,
      od.calendar_year AS order_year,
      o.paid_date_key,
      pd.day_name_of_week AS paid_day_name_of_week,
      pd.day_of_month AS paid_day_of_month,
      pd.weekday_weekend AS paid_weekday_weekend,
      pd.month_name AS paid_month_name,
      pd.calendar_quarter AS paid_calendar_quarter,
      pd.calendar_year AS paid_calendar_year,
      o.shipped_date_key,
      sd.day_name_of_week AS shipped_day_name_of_week,
      sd.day_of_month AS shipped_day_of_month,
      sd.weekday_weekend AS shipped_weekday_weekend,
      sd.month_name AS shipped_month_name,
      sd.calendar_quarter AS shipped_calendar_quarter,
      sd.calendar_year AS shipped_calendar_year,
      o.quantity,
      o.unit_price,
      o.discount,
      o.shipping_fee,
      o.taxes,
      o.tax_rate,
      o.payment_type,
      o.order_status,
      o.order_details_status
  FROM orders_silver_tempview AS o
  INNER JOIN northwind_dlh.dim_employee AS e
  ON e.employee_key = o.employee_key
  INNER JOIN northwind_dlh.dim_customer AS c
  ON c.customer_key = o.customer_key
  INNER JOIN northwind_dlh.dim_product AS p
  ON p.product_key = o.product_key
  INNER JOIN northwind_dlh.dim_shipper AS s
  ON s.shipper_key = o.shipper_key
  LEFT OUTER JOIN northwind_dlh.dim_date AS od
  ON od.date_key = o.order_date_key
  LEFT OUTER JOIN northwind_dlh.dim_date AS pd
  ON pd.date_key = o.paid_date_key
  LEFT OUTER JOIN northwind_dlh.dim_date AS sd
  ON sd.date_key = o.shipped_date_key
)

In [0]:
(spark.table("fact_orders_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{orders_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_silver"))

Out[41]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0db8787f70>

In [0]:
%sql
SELECT * FROM fact_orders_silver

fact_order_key,order_key,employee_key,employee_last_name,employee_first_name,employee_job_title,employee_company,customer_key,customer_last_name,customer_first_name,product_key,product_code,product_name,product_standard_cost,product_list_price,product_category,shipper_key,shipper_company,shipper_state_province,shipper_country_region,order_date_key,order_day_name_of_week,order_day_of_month,order_weekday_weekend,order_month_name,order_quarter,order_year,paid_date_key,paid_day_name_of_week,paid_day_of_month,paid_weekday_weekend,paid_month_name,paid_calendar_quarter,paid_calendar_year,shipped_date_key,shipped_day_name_of_week,shipped_day_of_month,shipped_weekday_weekend,shipped_month_name,shipped_calendar_quarter,shipped_calendar_year,quantity,unit_price,discount,shipping_fee,taxes,tax_rate,payment_type,order_status,order_details_status


In [0]:
%sql
DESCRIBE EXTENDED northwind_dlh.fact_orders_silver

col_name,data_type,comment
fact_order_key,bigint,null
order_key,bigint,null
employee_key,bigint,null
employee_last_name,string,null
employee_first_name,string,null
employee_job_title,string,null
employee_company,string,null
customer_key,bigint,null
customer_last_name,string,null
customer_first_name,string,null


##### 6.3. Gold Table: Perform Aggregations

In [0]:
%sql
SELECT customer_key AS CustomerID
  , customer_last_name AS LastName
  , customer_first_name AS FirstName
  , order_month_name AS OrderMonth
  , COUNT(product_key) AS ProductCount
FROM northwind_dlh.fact_orders_silver
GROUP BY CustomerID, LastName, FirstName, OrderMonth
ORDER BY ProductCount DESC

CustomerID,LastName,FirstName,OrderMonth,ProductCount
26,Liu,Run,April,3
8,Andersen,Elizabeth,April,3
6,Pérez-Olaeta,Francisco,June,3
9,Mortensen,Sven,April,2
6,Pérez-Olaeta,Francisco,April,2
11,Krschne,Peter,March,2
4,Lee,Christina,April,2
3,Axen,Thomas,April,2
10,Wacker,Roland,May,2
29,Lee,Soo Jung,April,1


In [0]:
%sql
SELECT pc.CustomerID
  , os.customer_last_name AS CustomerName
  , os.product_key AS ProductNumber
  , pc.ProductCount
FROM northwind_dlh.fact_orders_silver AS os
INNER JOIN (
  SELECT customer_key AS CustomerID
  , COUNT(product_key) AS ProductCount
  FROM northwind_dlh.fact_orders_silver
  GROUP BY customer_key
) AS pc
ON pc.CustomerID = os.customer_key
ORDER BY ProductCount DESC

CustomerID,CustomerName,ProductNumber,ProductCount
6,Pérez-Olaeta,34,5
6,Pérez-Olaeta,48,5
6,Pérez-Olaeta,7,5
6,Pérez-Olaeta,51,5
6,Pérez-Olaeta,48,5
8,Andersen,72,4
8,Andersen,8,4
8,Andersen,19,4
26,Liu,5,4
26,Liu,41,4


#### 7.0. Use AutoLoader to Process Streaming (Hot Path) Purchase Orders Fact Data 
##### 7.1. Bronze Table: Process 'Raw' JSON Data

In [0]:
# Use spark.readStream and the AutoLoader to read in the JSON files in the "purchase_orders_stream_dir"
# directory and then create a TempView named "purchase_orders_raw_tempview".
# Be sure to set the "cloudFiles.schemaLocation" Option using the "purchase_orders_output_bronze" directory
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 .option("cloudFiles.schemaHints", "fact_purchase_order_key BIGINT")
 .option("cloudFiles.schemaHints", "purchase_order_key BIGINT")
 .option("cloudFiles.schemaHints", "supplier_key BIGINT")
 .option("cloudFiles.schemaHints", "product_key BIGINT")
 .option("cloudFiles.schemaHints", "inventory_key DECIMAL")
 .option("cloudFiles.schemaHints", "created_by DECIMAL")
 .option("cloudFiles.schemaHints", "purchase_order_status BIGINT")
 .option("cloudFiles.schemaHints", "po_detail_posted_to_inventory BIGINT")
 .option("cloudFiles.schemaHints", "po_detail_unit_cost BIGINT")
 .option("cloudFiles.schemaHints", "po_detail_quantity BIGINT")
 .option("cloudFiles.schemaHints", "shipping_fee BIGINT")
 .option("cloudFiles.schemaHints", "taxes DECIMAL")
 .option("cloudFiles.schemaHints", "payment_date BIGINT")
 .option("cloudFiles.schemaHints", "payment_amount BIGINT")
 .option("cloudFiles.schemaHints", "approved_by BIGINT")
 .option("cloudFiles.schemaHints", "submitted_by BIGINT")
 .option("cloudFiles.schemaHints", "submitted_date_key DECIMAL")
 .option("cloudFiles.schemaHints", "creation_date_key DECIMAL")
 .option("cloudFiles.schemaHints", "po_detail_date_received_key DECIMAL")
 .option("cloudFiles.schemaHints", "approved_date_key DECIMAL")
 .option("cloudFiles.schemaLocation", purchase_orders_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(purchase_orders_stream_dir)
 .createOrReplaceTempView("purchase_orders_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW purchase_orders_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM purchase_orders_raw_tempview
)

In [0]:
%sql
SELECT * FROM purchase_orders_bronze_tempview

approved_by,approved_date_key,created_by,creation_date_key,fact_purchase_order_key,inventory_key,payment_amount,payment_date,po_detail_date_received_key,po_detail_posted_to_inventory,po_detail_quantity,po_detail_unit_cost,product_key,purchase_order_key,purchase_order_status,shipping_fee,submitted_by,submitted_date_key,supplier_key,taxes,_rescued_data,receipt_time,source_file
2,20060122,9,20060122,37,74,0,null,20060122,1,100,10.0,48,100,Approved,0,9,20060114,2,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060122,2,20060122,38,72,0,null,20060122,1,200,2.0,81,101,Approved,0,2,20060114,1,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,1,20060324,39,null,0,null,null,0,300,34.0,43,102,Approved,0,1,20060324,1,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,1,20060324,40,111,0,null,20060417,1,10,7.0,19,103,Approved,0,1,20060324,2,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,1,20060324,41,115,0,null,20060406,1,50,7.0,41,104,Approved,0,1,20060324,2,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,7,20060324,42,100,0,null,20060405,1,100,15.0,57,105,Approved,0,7,20060324,5,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,7,20060324,43,113,0,null,20060405,1,50,26.0,72,106,Approved,0,7,20060324,6,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,6,20060324,44,107,0,null,20060405,1,300,10.0,34,107,Approved,0,6,20060324,1,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,4,20060324,45,105,0,null,20060405,1,25,30.0,8,108,Approved,0,4,20060324,2,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,4,20060324,46,109,0,null,20060405,1,25,7.0,19,109,Approved,0,4,20060324,2,0,null,2023-04-24T18:13:14.029+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json


In [0]:
(spark.table("purchase_orders_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{purchase_orders_output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_purchase_orders_bronze"))

Out[49]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0db8774d60>

##### 7.2. Silver Table: Include Reference Data

In [0]:
(spark.readStream
  .table("fact_purchase_orders_bronze")
  .createOrReplaceTempView("purchase_orders_silver_tempview"))

In [0]:
%sql
SELECT * FROM purchase_orders_silver_tempview

approved_by,approved_date_key,created_by,creation_date_key,fact_purchase_order_key,inventory_key,payment_amount,payment_date,po_detail_date_received_key,po_detail_posted_to_inventory,po_detail_quantity,po_detail_unit_cost,product_key,purchase_order_key,purchase_order_status,shipping_fee,submitted_by,submitted_date_key,supplier_key,taxes,_rescued_data,receipt_time,source_file
2,20060122,9,20060122,37,74,0,null,20060122,1,100,10.0,48,100,Approved,0,9,20060114,2,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060122,2,20060122,38,72,0,null,20060122,1,200,2.0,81,101,Approved,0,2,20060114,1,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,1,20060324,39,null,0,null,null,0,300,34.0,43,102,Approved,0,1,20060324,1,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,1,20060324,40,111,0,null,20060417,1,10,7.0,19,103,Approved,0,1,20060324,2,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,1,20060324,41,115,0,null,20060406,1,50,7.0,41,104,Approved,0,1,20060324,2,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,7,20060324,42,100,0,null,20060405,1,100,15.0,57,105,Approved,0,7,20060324,5,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,7,20060324,43,113,0,null,20060405,1,50,26.0,72,106,Approved,0,7,20060324,6,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,6,20060324,44,107,0,null,20060405,1,300,10.0,34,107,Approved,0,6,20060324,1,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,4,20060324,45,105,0,null,20060405,1,25,30.0,8,108,Approved,0,4,20060324,2,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json
2,20060404,4,20060324,46,109,0,null,20060405,1,25,7.0,19,109,Approved,0,4,20060324,2,0,null,2023-04-24T18:13:17.103+0000,dbfs:/FileStore/ds2002-lab06/source_data/stream/purchase_orders/Northwind_Fact_PurchaseOrders03.json


In [0]:
%sql
DESCRIBE EXTENDED purchase_orders_silver_tempview

col_name,data_type,comment
approved_by,bigint,null
approved_date_key,"decimal(10,0)",null
created_by,bigint,null
creation_date_key,bigint,null
fact_purchase_order_key,bigint,null
inventory_key,bigint,null
payment_amount,bigint,null
payment_date,string,null
po_detail_date_received_key,bigint,null
po_detail_posted_to_inventory,bigint,null


In [0]:
%sql
-- Create a new Temporary View named "purchase_orders_silver_tempview" by selecting data from
-- "purchase_orders_silver_tempview" and joining it to the Supplier, Product, Employee and Data dimension tables.
-- Remember that the Date dimension can serve as a "Role Playing" dimension by being Joined upon multiple times.
CREATE OR REPLACE TEMPORARY VIEW fact_purchase_orders_silver_tempview AS (
  SELECT o.fact_purchase_order_key,
      o.purchase_order_key,
      o.supplier_key,
      s.company AS supplier_company,
      s.last_name AS supplier_last_name,
      s.first_name AS supplier_first_name,
      o.product_key,
      p.product_code,
      p.product_name,
      o.submitted_date_key,
      p.standard_cost AS product_standard_cost,
      p.list_price AS product_list_price,
      p.category AS product_category,
      od.day_name_of_week AS order_day_name_of_week,
      od.day_of_month AS order_day_of_month,
      od.weekday_weekend AS order_weekday_weekend,
      od.month_name AS order_month_name,
      od.calendar_quarter AS order_quarter,
      od.calendar_year AS order_year
  FROM purchase_orders_silver_tempview AS o
  INNER JOIN northwind_dlh.dim_product AS p
  ON p.product_key = o.product_key
  INNER JOIN northwind_dlh.dim_supplier AS s
  ON s.supplier_key = o.supplier_key
  LEFT OUTER JOIN northwind_dlh.dim_date AS od
  ON od.date_key = o.submitted_date_key
)

In [0]:
(spark.table("fact_purchase_orders_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{purchase_orders_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_purchase_orders_silver"))

Out[54]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0dc0141580>

In [0]:
%sql
SELECT * FROM fact_purchase_orders_silver

fact_purchase_order_key,purchase_order_key,supplier_key,supplier_company,supplier_last_name,supplier_first_name,product_key,product_code,product_name,submitted_date_key,product_standard_cost,product_list_price,product_category,order_day_name_of_week,order_day_of_month,order_weekday_weekend,order_month_name,order_quarter,order_year


In [0]:
%sql
DESCRIBE EXTENDED fact_purchase_orders_silver

col_name,data_type,comment
fact_purchase_order_key,bigint,null
purchase_order_key,bigint,null
supplier_key,bigint,null
supplier_company,string,null
supplier_last_name,string,null
supplier_first_name,string,null
product_key,bigint,null
product_code,string,null
product_name,string,null
submitted_date_key,bigint,null


##### 7.3. Gold Table: Perform Aggregations

In [0]:
%sql
-- Author a query that returns the Total List Price grouped by Supplier and Product and sorted by Total List Price descending.
SELECT supplier_company AS Supplier
  , product_name AS Product
  , product_list_price AS Total_List_Price
FROM northwind_dlh.fact_purchase_orders_silver
GROUP BY Supplier, Product, Total_List_Price
ORDER BY Total_List_Price DESC

Supplier,Product,Total_List_Price
Supplier B,Northwind Traders Marmalade,81.0000
Supplier B,Northwind Traders Dried Apples,53.0000
Supplier A,Northwind Traders Coffee,46.0000
Supplier B,Northwind Traders Curry Sauce,40.0000
Supplier B,Northwind Traders Fruit Cocktail,39.0000
Supplier E,Northwind Traders Gnocchi,38.0000
Supplier E,Northwind Traders Mozzarella,34.8000
Supplier F,Northwind Traders Mozzarella,34.8000
Supplier B,Northwind Traders Dried Pears,30.0000
Supplier H,Northwind Traders Boysenberry Spread,25.0000


#### 8.0. Use AutoLoader to Process Streaming (Hot Path) Inventory Transactions Fact Data 
##### 8.1. Bronze Table: Process 'Raw' JSON Data

In [0]:
# Use spark.readStream and the AutoLoader to read in the JSON files in the "inventory_trans_stream_dir"
# directory and then create a TempView named "inventory_transactions_raw_tempview".
# Be sure to set the "cloudFiles.schemaLocation" Option using the "inventory_trans_output_bronze" directory
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 .option("cloudFiles.schemaHints", "fact_inventory_transaction_key BIGINT")
 .option("cloudFiles.schemaHints", "inventory_transaction_key BIGINT")
 .option("cloudFiles.schemaHints", "product_key BIGINT")
 .option("cloudFiles.schemaHints", "transaction_created_date_key DECIMAL")
 .option("cloudFiles.schemaHints", "transaction_modified_date_key DECIMAL")
 .option("cloudFiles.schemaHints", "inventory_transaction_type STRING")
 .option("cloudFiles.schemaHints", "quantity DECIMAL")
 .option("cloudFiles.schemaHints", "purchase_order_key BIGINT")
 .option("cloudFiles.schemaHints", "order_key BIGINT")
 .option("cloudFiles.schemaLocation", inventory_trans_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(inventory_trans_stream_dir)
 .createOrReplaceTempView("inventory_transactions_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW inventory_transactions_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM inventory_transactions_raw_tempview
)

In [0]:
%sql
SELECT * FROM inventory_transactions_raw_tempview

fact_inventory_transaction_key,inventory_transaction_key,inventory_transaction_type,order_key,product_key,purchase_order_key,quantity,transaction_created_date_key,transaction_modified_date_key,_rescued_data


In [0]:
(spark.table("inventory_transactions_bronze_tempview")
     .writeStream
     .format("delta")
     .option("checkpointLocation", f"{inventory_trans_output_bronze}/_checkpoint")
     .outputMode("append")
     .table("fact_inventory_transactions_bronze"))

Out[61]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0db8774f40>

##### 8.2. Silver Table: Include Reference Data

In [0]:
(spark.readStream
  .table("fact_inventory_transactions_bronze")
  .createOrReplaceTempView("inventory_transactions_silver_tempview"))

In [0]:
%sql
SELECT * FROM inventory_transactions_silver_tempview

fact_inventory_transaction_key,inventory_transaction_key,inventory_transaction_type,order_key,product_key,purchase_order_key,quantity,transaction_created_date_key,transaction_modified_date_key,_rescued_data,receipt_time,source_file


In [0]:
%sql
DESCRIBE EXTENDED inventory_transactions_silver_tempview

col_name,data_type,comment
fact_inventory_transaction_key,bigint,null
inventory_transaction_key,bigint,null
inventory_transaction_type,string,null
order_key,bigint,null
product_key,bigint,null
purchase_order_key,string,null
quantity,bigint,null
transaction_created_date_key,bigint,null
transaction_modified_date_key,bigint,null
_rescued_data,string,null


In [0]:
%sql
-- Create a new Temporary View named "fact_inventory_transactions_silver_tempview" by selecting data from
-- "inventory_transactions_silver_tempview" and joining it to the Product and Data dimension tables.
-- Remember that the Date dimension can serve as a "Role Playing" dimension by being Joined upon multiple times.
CREATE OR REPLACE TEMPORARY VIEW fact_inventory_transactions_silver_tempview AS (
  SELECT o.fact_inventory_transaction_key,
      o.inventory_transaction_key,
      o.inventory_transaction_type,
      o.product_key,
      o.transaction_created_date_key,
      o.quantity,
      o.order_key,
      p.product_code,
      p.product_name,
      p.standard_cost AS product_standard_cost,
      p.list_price AS product_list_price,
      p.category AS product_category,
      od.day_name_of_week AS order_day_name_of_week,
      od.day_of_month AS order_day_of_month,
      od.weekday_weekend AS order_weekday_weekend,
      od.month_name AS order_month_name,
      od.calendar_quarter AS order_quarter,
      od.calendar_year AS order_year
  FROM inventory_transactions_silver_tempview AS o
  INNER JOIN northwind_dlh.dim_product AS p
  ON p.product_key = o.product_key
  LEFT OUTER JOIN northwind_dlh.dim_date AS od
  ON od.date_key = o.transaction_created_date_key
)

In [0]:
(spark.table("fact_inventory_transactions_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{inventory_trans_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_inventory_transactions_silver"))

Out[74]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f0db8706610>

In [0]:
%sql
SELECT * FROM fact_inventory_transactions_silver

fact_inventory_transaction_key,inventory_transaction_key,inventory_transaction_type,product_key,transaction_created_date_key,quantity,order_key,product_code,product_name,product_standard_cost,product_list_price,product_category,order_day_name_of_week,order_day_of_month,order_weekday_weekend,order_month_name,order_quarter,order_year
1,35,Purchased,80,20060322,75,null,NWTDFN-80,Northwind Traders Dried Plums,3.0000,3.5000,Dried Fruit & Nuts,Wednesday,22,Weekday,March,1,2006
2,36,Purchased,72,20060322,40,null,NWTD-72,Northwind Traders Mozzarella,26.1000,34.8000,Dairy products,Wednesday,22,Weekday,March,1,2006
3,37,Purchased,52,20060322,100,null,NWTG-52,Northwind Traders Long Grain Rice,5.2500,7.0000,Grains,Wednesday,22,Weekday,March,1,2006
4,38,Purchased,56,20060322,120,null,NWTP-56,Northwind Traders Gnocchi,28.5000,38.0000,Pasta,Wednesday,22,Weekday,March,1,2006
5,39,Purchased,57,20060322,80,null,NWTP-57,Northwind Traders Ravioli,14.6250,19.5000,Pasta,Wednesday,22,Weekday,March,1,2006
6,40,Purchased,6,20060322,100,null,NWTJP-6,Northwind Traders Boysenberry Spread,18.7500,25.0000,"Jams, Preserves",Wednesday,22,Weekday,March,1,2006
7,41,Purchased,7,20060322,40,null,NWTDFN-7,Northwind Traders Dried Pears,22.5000,30.0000,Dried Fruit & Nuts,Wednesday,22,Weekday,March,1,2006
8,42,Purchased,8,20060322,40,null,NWTS-8,Northwind Traders Curry Sauce,30.0000,40.0000,Sauces,Wednesday,22,Weekday,March,1,2006
9,43,Purchased,14,20060322,40,null,NWTDFN-14,Northwind Traders Walnuts,17.4375,23.2500,Dried Fruit & Nuts,Wednesday,22,Weekday,March,1,2006
10,44,Purchased,17,20060322,40,null,NWTCFV-17,Northwind Traders Fruit Cocktail,29.2500,39.0000,Canned Fruit & Vegetables,Wednesday,22,Weekday,March,1,2006


In [0]:
%sql
DESCRIBE EXTENDED fact_inventory_transactions_silver

col_name,data_type,comment
fact_inventory_transaction_key,bigint,null
inventory_transaction_key,bigint,null
inventory_transaction_type,string,null
product_key,bigint,null
transaction_created_date_key,bigint,null
quantity,bigint,null
order_key,bigint,null
product_code,string,null
product_name,string,null
product_standard_cost,"decimal(19,4)",null


##### 8.3. Gold Table: Perform Aggregations

In [0]:
%sql
-- Author a query that returns the Total Quantity grouped by the Quarter Created, Inventory Transaction Type, and Product
-- Sort by the Total Quantity Descending
SELECT quantity AS Total_Quantity
  , order_quarter AS Quarter_Created
  , inventory_transaction_type AS Inventory_Transaction_Type
  , product_name AS Product
FROM northwind_dlh.fact_inventory_transactions_silver
GROUP BY Product, Quarter_Created, Inventory_Transaction_Type, Total_Quantity
ORDER BY Total_Quantity DESC

Total_Quantity,Quarter_Created,Inventory_Transaction_Type,Product
300,1,Purchased,Northwind Traders Coffee
300,2,Sold,Northwind Traders Beer
300,2,On Hold,Northwind Traders Coffee
300,1,Sold,Northwind Traders Coffee
300,2,Purchased,Northwind Traders Beer
250,2,Purchased,Northwind Traders Coffee
200,1,Purchased,Northwind Traders Green Tea
200,1,Purchased,Northwind Traders Clam Chowder
200,1,Sold,Northwind Traders Green Tea
200,1,Sold,Northwind Traders Clam Chowder


#### 9.0. Clean up the File System

In [0]:
%fs rm -r /FileStore/ds2002-lab06/

res12: Boolean = true